In [34]:
import pandas as pd
import numpy as np
import json

import gensim
from gensim import corpora, models
from gensim.models import CoherenceModel

import time
import pickle
from tqdm import tqdm

import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

from konlpy.tag import Okt

from collections import Counter

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [10]:
with open('review_jsn_147323.json','r') as fp:
    review = json.load(fp)

In [18]:
review['18551259'][0]

{'collection_tag': None,
 'comments': '새벽이 도착했음에도 편안히 잘 쉴수 있었던 공간이었습니다. 반갑게 맞아준 헨리도 기억에 남네요. 너무 늦게 들어가서 놀아줄 시간이 없어 아쉬웠어요^^',
 'created_at': '2018-10-06T06:15:10Z',
 'id': 332696014,
 'language': 'ko',
 'localized_date': '2018년 10월',
 'rating': 5,
 'response': '',
 'reviewee': {'deleted': False,
  'first_name': 'David',
  'host_name': 'David',
  'id': 76010772,
  'is_superhost': False,
  'picture_url': 'https://a0.muscache.com/im/pictures/user/09c053d6-07a4-4c0e-a864-7fce8458baf6.jpg?aki_policy=profile_x_medium',
  'profile_path': '/users/show/76010772'},
 'reviewer': {'deleted': False,
  'first_name': 'Junghee',
  'host_name': 'Junghee',
  'id': 111507566,
  'is_superhost': False,
  'picture_url': 'https://a0.muscache.com/im/pictures/c1936b91-a6f7-4702-9db6-48df6ce827f9.jpg?aki_policy=profile_x_medium',
  'profile_path': '/users/show/111507566'}}

In [13]:
okt = Okt() # Twitter 형태소분석기와 같음

In [20]:
corpus = []
for listing_id, rev_lst in review.items():
    for rev in rev_lst:
        corpus.append(rev['comments'])

In [21]:
len(corpus)

160716

In [26]:
corpus_nouns = [okt.nouns(comment) for comment in corpus] 

In [29]:
dic = corpora.Dictionary(corpus_nouns)

In [30]:
len(dic) #출현 단어 개수

12628

In [31]:
corpus_vector = [dic.doc2bow(doc) for doc in corpus_nouns] #단어별 출현 횟수 vector 구성

In [33]:
corpus[1]

'개인공간에 대한 프라이버시 보장이 되어 좋았고 개인 룸은 깨끗하고 생각보다 넓어서 좋았습니다. '

In [32]:
corpus_vector[1]

[(0, 1), (7, 2), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1)]

In [36]:
%%time
ldamodel = gensim.models.ldamodel.LdaModel(
                        corpus_vector, 
                        num_topics=5, 
                        id2word = dic, 
                        passes=10
                        ) #LDA model 생성

CPU times: user 2min 32s, sys: 41.6 ms, total: 2min 32s
Wall time: 2min 32s


In [37]:
ldamodel.show_topics(num_words=10) #토픽을 구성하는 주요단어들

[(0,
  '0.103*"위치" + 0.082*"숙소" + 0.038*"정말" + 0.038*"시설" + 0.037*"매우" + 0.026*"호스트" + 0.024*"주변" + 0.021*"아주" + 0.020*"편의" + 0.018*"가격"'),
 (1,
  '0.017*"것" + 0.017*"침대" + 0.017*"조금" + 0.016*"좀" + 0.015*"화장실" + 0.014*"방" + 0.014*"사진" + 0.013*"수건" + 0.013*"다만" + 0.012*"점"'),
 (2,
  '0.057*"또" + 0.051*"다음" + 0.047*"이용" + 0.030*"정말" + 0.030*"숙소" + 0.025*"곳" + 0.020*"다시" + 0.019*"서울" + 0.015*"방문" + 0.015*"호스트"'),
 (3,
  '0.052*"바로" + 0.033*"편의점" + 0.027*"근처" + 0.024*"앞" + 0.021*"위치" + 0.018*"주변" + 0.017*"숙소" + 0.016*"거리" + 0.014*"집" + 0.014*"접근성"'),
 (4,
  '0.050*"호스트" + 0.036*"숙소" + 0.022*"것" + 0.018*"시간" + 0.018*"정말" + 0.018*"배려" + 0.018*"수" + 0.016*"위치" + 0.015*"체크" + 0.015*"연락"')]

In [38]:
# Compute Perplexity 
print('\nPerplexity: ', ldamodel.log_perplexity(corpus_vector))  # a measure of how good the model is. lower the better.


Perplexity:  -6.9880433663119454


In [39]:
# Compute Coherence Score 
coherence_model_lda = CoherenceModel(model=ldamodel, texts=corpus_nouns, dictionary=dic, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.5785306226855491


In [40]:
# Visualize the topics

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(ldamodel, corpus_vector, dic)
vis

/home/chankoo/anaconda3/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.250499 -0.235576       1        1  25.813377
4      0.131565  0.117230       2        1  21.689787
2      0.054139  0.195979       3        1  20.212984
0     -0.158081  0.094856       4        1  16.307894
3     -0.278123 -0.172489       5        1  15.975955, topic_info=     Category          Freq   Term         Total  loglift  logprob
term                                                              
39    Default  14410.000000     위치  14410.000000  30.0000  30.0000
48    Default   6987.000000      또   6987.000000  29.0000  29.0000
47    Default   6246.000000     다음   6246.000000  28.0000  28.0000
543   Default   5021.000000     바로   5021.000000  27.0000  27.0000
24    Default  18228.000000     숙소  18228.000000  26.0000  26.0000
58    Default  11027.000000    호스트  11027.000000  25.0000  25.0000
240   Default   3735.000000     시설   3735.000000  24.0000  24.0000
54    Default   7274.000000     이용   7274.000000  23.0000  23.0000
34    Default   9747.000000     정말   9747.000000  22.0000  22.0000
233   Default   4048.000000     매우   4048.000000  21.0000  21.0000
466   Default   3645.000000    편의점   3645.000000  20.0000  20.0000
221   Default   4073.000000     주변   4073.000000  19.0000  19.0000
290   Default   2359.000000      앞   2359.000000  18.0000  18.0000
228   Default   3649.000000     근처   3649.000000  17.0000  17.0000
120   Default   2488.000000     다시   2488.000000  16.0000  16.0000
302   Default   2294.000000     서울   2294.000000  15.0000  15.0000
250   Default   1990.000000     편의   1990.000000  14.0000  14.0000
538   Default   2334.000000     배려   2334.000000  13.0000  13.0000
35    Default   2487.000000      좀   2487.000000  12.0000  12.0000
292   Default   2837.000000      역   2837.000000  11.0000  11.0000
402   Default   2288.000000    화장실   2288.000000  10.0000  10.0000
267   Default   2015.000000     체크   2015.000000   9.0000   9.0000
25    Default   2077.000000     가격   2077.000000   8.0000   8.0000
4     Default   2864.000000     시간   2864.000000   7.0000   7.0000
544   Default   1943.000000     연락   1943.000000   6.0000   6.0000
40    Default   3340.000000     조금   3340.000000   5.0000   5.0000
13    Default   1994.000000     다만   1994.000000   4.0000   4.0000
384   Default   2167.000000     교통   2167.000000   3.0000   3.0000
599   Default   1747.000000     답변   1747.000000   2.0000   2.0000
671   Default   1733.000000     세심   1733.000000   1.0000   1.0000
...       ...           ...    ...           ...      ...      ...
743    Topic5    393.694061     산책    394.429291   1.8322  -5.5077
697    Topic5    375.271515  홍대입구역    376.011200   1.8321  -5.5556
242    Topic5    375.503510     장점    376.247101   1.8321  -5.5550
1057   Topic5    354.504303     중심    355.241272   1.8320  -5.6125
2364   Topic5    352.673828     아래    353.417297   1.8320  -5.6177
2098   Topic5    339.677917     공항    340.416962   1.8319  -5.6553
201    Topic5    339.102997     파티    339.842773   1.8319  -5.6570
717    Topic5    328.245911     강추    328.992065   1.8318  -5.6895
1574   Topic5    316.062317     적극    316.802216   1.8317  -5.7273
577    Topic5    306.325256     택시    307.069733   1.8317  -5.7586
906    Topic5    301.560455     호선    302.293823   1.8317  -5.7743
466    Topic5   3211.221191    편의점   3645.728516   1.7072  -3.4089
228    Topic5   2589.674316     근처   3649.931641   1.4909  -3.6240
837    Topic5   1526.948242     거리   2096.994141   1.5169  -4.1522
937    Topic5    810.284546      옆   1000.199097   1.6235  -4.7859
326    Topic5    879.022888    지하철   1262.147583   1.4723  -4.7045
221    Topic5   1717.903442     주변   4073.561523   0.9707  -4.0344
89     Topic5   1398.851685      집   4287.775391   0.7140  -4.2399
146    Topic5    761.236450      번   1465.625977   1.1790  -4.8483
292    Topic5   1065.123413      역   2837.241943   0.8543  -4.5124
196    Topic5   1081.412231 